 <h1 style="text-align: center; color: RED">BG-NBD VE GAMMA-GAMMA İLE CLTV TAHMİNİ</h1>

<span style="color: blue; font-weight: bold;">İŞ PROBLEMİ</span> 

FLO satış ve pazarlama faaliyetleri için roadmap
belirlemek istemektedir. Şirketin orta uzun vadeli plan
yapabilmesi için var olan müşterilerin gelecekte şirkete
sağlayacakları potansiyel değerin tahmin edilmesi
gerekmektedir.

<span style="color: blue; font-weight: bold;">VERİ SETİ HİKAYESİ</span> 

Veri seti son alışverişlerini 2020 - 2021 yıllarında OmniChannel(hem online hem offline alışveriş yapan) olarak yapan müşterilerin geçmiş alışveriş davranışlarından
elde edilen bilgilerden oluşmaktadır.

* master_id: Eşsiz müşteri numarası
* order_channel : Alışveriş yapılan platforma ait hangi kanalın kullanıldığı (Android, ios, Desktop, Mobile, Offline)
* last_order_channel : En son alışverişin yapıldığı kanal
* first_order_date : Müşterinin yaptığı ilk alışveriş tarihi
* last_order_date : Müşterinin yaptığı son alışveriş tarihi
* last_order_date_online : Muşterinin online platformda yaptığı son alışveriş tarihi
* last_order_date_offline : Muşterinin offline platformda yaptığı son alışveriş tarihi
* order_num_total_ever_online : Müşterinin online platformda yaptığı toplam alışveriş sayısı
* order_num_total_ever_offline : Müşterinin offline'da yaptığı toplam alışveriş sayısı
* customer_value_total_ever_offline : Müşterinin offline alışverişlerinde ödediği toplam ücret
* customer_value_total_ever_online : Müşterinin online alışverişlerinde ödediği toplam ücret
* interested_in_categories_12 : Müşterinin son 12 ayda alışveriş yaptığı kategorilerin listesi

<span style="color: blue; font-weight: bold;">GÖREV1: VERİYİ HAZIRLAMA</span> 

In [15]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)


**Adım1:** flo_data_20K.csv verisini okuyunuz.

In [73]:
df_ = pd.read_csv("C:\\Users\\ASLI\\Desktop\\Data Scientist Bootcamp\\3.CRM Analytics\\resim\\flo_data_20k.csv")

In [208]:
df = df_.copy()

**Adım2:** Aykırı değerleri baskılamak için gerekli olan outlier_thresholds ve replace_with_thresholds fonksiyonlarını tanımlayınız.
Not: cltv hesaplanırken frequency değerleri integer olması gerekmektedir.Bu nedenle alt ve üst limitlerini round() ile yuvarlayınız.

In [212]:
def outlier_thresholds(dataframe,variable):
    q1 = dataframe[variable].quantile(0.01)
    q3 = dataframe[variable].quantile(0.99)
    IQR = q3 - q1
    low_limit = q1 - IQR*1.5
    up_limit = q3 + IQR*1.5
    return low_limit,up_limit

In [214]:
def replace_with_thresholds(dataframe,variable):
    low_limit, up_limit = outlier_thresholds(dataframe,variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit,0)
    dataframe.loc[(dataframe[variable] > up_limit) , variable] = round(up_limit,0)

**Adım3:** "order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline",
"customer_value_total_ever_online" değişkenlerinin aykırı değerleri varsa baskılayanız.

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [216]:
num_col = [col for col in df. columns if df[col].dtype !='O']

In [56]:
num_col

['order_num_total_ever_online',
 'order_num_total_ever_offline',
 'customer_value_total_ever_offline',
 'customer_value_total_ever_online']

In [210]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.0000,3.1109,4.2256,1.0000,1.0000,2.0000,4.0000,200.0000
order_num_total_ever_offline,19945.0000,1.9139,2.0629,1.0000,1.0000,1.0000,2.0000,109.0000
customer_value_total_ever_offline,19945.0000,253.9226,301.5329,10.0000,99.9900,179.9800,319.9700,18119.1400
customer_value_total_ever_online,19945.0000,497.3217,832.6019,12.9900,149.9800,286.4600,578.4400,45220.1300


In [218]:
for col in num_col:
    print(outlier_thresholds(df,col))

(-27.5, 48.5)
(-8.0, 16.0)
(-1779.9452000000003, 3019.8820000000005)
(-4615.7406000000055, 7799.541000000008)


In [220]:
for col in num_col:
    replace_with_thresholds(df,col)

In [222]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.0000,3.0920,3.8095,1.0000,1.0000,2.0000,4.0000,48.0000
order_num_total_ever_offline,19945.0000,1.8862,1.4347,1.0000,1.0000,1.0000,2.0000,16.0000
customer_value_total_ever_offline,19945.0000,251.9213,251.0237,10.0000,99.9900,179.9800,319.9700,3020.0000
customer_value_total_ever_online,19945.0000,489.7057,632.6098,12.9900,149.9800,286.4600,578.4400,7800.0000


**Adım4:** Omnichannel müşterilerin hem online'dan hem de offline platformlardan alışveriş yaptığını ifade etmektedir. Her bir müşterinin toplam alışveriş sayısı ve harcaması için yeni değişkenler oluşturunuz.

In [224]:
df['total_order_num'] = df['order_num_total_ever_online'] + df['order_num_total_ever_offline']

In [226]:
df['total_customer_value'] = df['customer_value_total_ever_offline'] + df['customer_value_total_ever_online']

In [228]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_order_num,total_customer_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN],5.0000,939.3700
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0000,2.0000,159.9700,1853.5800,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0000,2013.5500
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]",5.0000,585.3200
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]",2.0000,121.9700
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR],2.0000,209.9800


**Adım5:** Değişken tiplerini inceleyiniz. Tarih ifade eden değişkenlerin tipini date'e çeviriniz.

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [230]:
date_col = df.columns[df.columns.str.contains('date', na=False)]

In [106]:
date_col

Index(['first_order_date', 'last_order_date', 'last_order_date_online', 'last_order_date_offline'], dtype='object')

In [232]:
df[date_col] = df[date_col].apply(pd.to_datetime)

In [234]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

<span style="color: blue; font-weight: bold;">GÖREV2: CLTV VERİ YAPISININ OLUŞTURULMASI</span> 

**Adım1:** Veri setindeki en son alışverişin yapıldığı tarihten 2 gün sonrasını analiz tarihi olarak alınız.

In [118]:
df['last_order_date'].max()

Timestamp('2021-05-30 00:00:00')

In [260]:
analys_date = dt.datetime(2021,6,1) #model kurmada hata verdiği için tarihi böyle sectik

**Adım2:** customer_id, recency_cltv_weekly, T_weekly, frequency ve monetary_cltv_avg değerlerinin yer aldığı yeni bir cltv dataframe'i
oluşturunuz. Monetary değeri satın alma başına ortalama değer olarak, recency ve tenure değerleri ise haftalık cinsten ifade edilecek.

Bu kısımda bg/nbd ve gamma gamma modelinin istediği yapıyı oluşturmayı çalışıyoruz.

* recency: Son satın alma üzerinden geçen zaman. Haftalık. (kullanıcı özelinde) Bu analiz tarihine göre değil kullanıcının ilk alışveriş - son alışveriş tarihine göe belirlerim.
* T: Müşterinin yaşı. Haftalık. (analiz tarihinden ne kadar süre önce ilk satın alma yapılmış)
* frequency: tekrar eden toplam satın alma sayısı (frequency>1)
* monetary: satın alma başına ortalama kazanç

In [388]:
cltv = pd.DataFrame()

In [390]:
cltv['customer_id'] = df['master_id']

In [392]:
cltv['recency'] = ((df['last_order_date'] - df['first_order_date']).dt.days)/7

In [394]:
cltv['T'] = ((analys_date - df['first_order_date']).dt.days)/7

In [396]:
cltv['frequency'] = df['total_order_num'] 

In [398]:
cltv['monetary'] = df['total_customer_value'] / df['total_order_num']

In [400]:
cltv = cltv[cltv['frequency'] > 1]

In [402]:
cltv.head(10)

,customer_id,recency,T,frequency,monetary
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,120.8571,132.2857,3.0000,66.9533
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,32.5714,64.8571,4.0000,93.9825
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,12.7143,54.5714,2.0000,81.8150
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,58.4286,70.7143,5.0000,210.9380
9,1143f032-440d-11ea-8b43-000d3a38a36f,61.7143,96.0000,2.0000,82.9800


<span style="color: blue; font-weight: bold;">GÖREV3: BG/NBD VE GAMMA GAMMA MODELLERİNİN KURULMASI</span> 

**Adım 1 :** BG/NBD modelini fit ediniz.

3 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz ve exp_sales_3_month olarak cltv dataframe'ine ekleyiniz.
              
6 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz ve exp_sales_6_month olarak cltv dataframe'ine ekleyiniz.


In [406]:
bgf = BetaGeoFitter(penalizer_coef = 0.001)

In [408]:
bgf.fit(cltv['frequency'],
        cltv['recency'],
        cltv['T'])

C:\Users\ASLI\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.17, b: 0.00, r: 3.66>

In [410]:
cltv['expected_sales_3_month'] = bgf.predict(4*3,
                                            cltv['frequency'],
                                            cltv['recency'],
                                            cltv['T'])

In [412]:
cltv['expected_sales_6_month'] = bgf.predict(6*4,
                                            cltv['frequency'],
                                            cltv['recency'],
                                            cltv['T'])

In [414]:
cltv.head()

,customer_id,recency,T,frequency,monetary,expected_sales_3_month,expected_sales_6_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740,0.9739,1.9479
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833,0.9832,1.9663
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640,0.6706,1.3412
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850,0.7004,1.4008
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900,0.3960,0.7921


**Adım2:** Gamma-Gamma modelini fit ediniz. Müşterilerin ortalama bırakacakları değeri tahminleyip exp_average_value olarak cltv dataframe'ine ekleyiniz.

In [417]:
ggf = GammaGammaFitter(penalizer_coef=0.001)

In [419]:
ggf.fit(cltv['frequency'],cltv['monetary'])

<lifetimes.GammaGammaFitter: fitted with 19945 subjects, p: 13.19, q: 1.70, v: 12.96>

In [421]:
cltv['exp_average_value'] = ggf.conditional_expected_average_profit(cltv['frequency'],cltv['monetary'])

In [423]:
cltv.head()

,customer_id,recency,T,frequency,monetary,expected_sales_3_month,expected_sales_6_month,exp_average_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740,0.9739,1.9479,188.4700
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833,0.9832,1.9663,96.2577
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640,0.6706,1.3412,118.4019
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850,0.7004,1.4008,65.7243
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900,0.3960,0.7921,108.5945


**Adım3:** 6 aylık CLTV hesaplayınız ve cltv ismiyle dataframe'e ekleyiniz. Cltv değeri en yüksek 20 kişiyi gözlemleyiniz.

In [426]:
clt = ggf.customer_lifetime_value(bgf,
                                  cltv['frequency'],
                                  cltv['recency'],
                                  cltv['T'],
                                  cltv['monetary'],
                                  time = 6,
                                  freq = 'W',
                                  discount_rate = 0.01)

In [428]:
cltv['cltv'] = clt

In [430]:
cltv.head()

,customer_id,recency,T,frequency,monetary,expected_sales_3_month,expected_sales_6_month,exp_average_value,cltv
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740,0.9739,1.9479,188.4700,385.1821
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833,0.9832,1.9663,96.2577,198.5903
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640,0.6706,1.3412,118.4019,166.6138
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850,0.7004,1.4008,65.7243,96.6000
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900,0.3960,0.7921,108.5945,90.2491


In [436]:
cltv.sort_values('cltv', ascending=False).head(20)

,customer_id,recency,T,frequency,monetary,expected_sales_3_month,expected_sales_6_month,exp_average_value,cltv
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.8571,7.8571,4.0000,1401.8000,1.0944,2.1888,1386.6859,3184.5334
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.1429,13.1429,11.0000,758.0855,1.9701,3.9402,755.6267,3123.8844
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.7143,101.0000,7.0000,1106.4671,0.7222,1.4445,1099.9988,1667.1328
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.2857,74.5714,16.0000,501.8737,1.5653,3.1306,501.0258,1645.7272
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.7143,67.2857,52.0000,166.2246,4.6561,9.3123,166.3045,1624.9033
8868,9ce6e520-89b0-11ea-a6e7-000d3a38a36f,3.4286,34.4286,8.0000,601.2262,1.2655,2.5309,598.8827,1590.3273
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.7143,13.0000,17.0000,259.8653,2.7807,5.5614,259.8184,1516.0726
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.5714,58.7143,31.0000,228.5300,3.0838,6.1676,228.5577,1479.0291
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8.2857,9.4286,2.0000,862.6900,0.7939,1.5878,846.7544,1410.6976
4157,7eed6468-4540-11ea-acaf-000d3a38a36f,89.1429,90.0000,27.0000,289.7641,2.2143,4.4287,289.6760,1346.0238


<span style="color: blue; font-weight: bold;">GÖREV4: CLTV DEĞERİNE GÖRE SEGMENTLERİN OLUŞTURULMASI</span> 

**Adım1:** 6 aylık CLTV'ye göre tüm müşterilerinizi 4 gruba (segmente) ayırınız ve grup isimlerini veri setine ekleyiniz.

In [443]:
cltv['segment'] = pd.qcut(cltv['cltv'],4, labels = ['D','C','B','A'])

In [445]:
cltv.head()

,customer_id,recency,T,frequency,monetary,expected_sales_3_month,expected_sales_6_month,exp_average_value,cltv,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740,0.9739,1.9479,188.4700,385.1821,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833,0.9832,1.9663,96.2577,198.5903,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640,0.6706,1.3412,118.4019,166.6138,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850,0.7004,1.4008,65.7243,96.6000,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900,0.3960,0.7921,108.5945,90.2491,D
